In [1]:
import os

%pwd

'c:\\Personal Projects\\NephroMassSorter\\notebook'

In [2]:
os.chdir("../")

%pwd

'c:\\Personal Projects\\NephroMassSorter'

In [3]:
!ls -a

.
..
.dvc
.dvcignore
.git
.github
.gitignore
Dockerfile
LICENSE
README.md
app.py
config
dvc.yaml
logs
main.py
notebook
params.yaml
requirements.txt
setup.py
src
template.py
templates


### Updating config_entity.py in the entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Update the configuration manager, configuration.py in the config

In [5]:
from claSSifier.constants import *
from claSSifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Update data_ingestion.py in the components

In [6]:
import os
import zipfile
import gdown
from claSSifier import logger
from claSSifier.utils.common import get_size


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### Update pipeline: stage_01_data_ingestion.py is updated

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-28 11:53:07,319: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-28 11:53:07,336: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-28 11:53:07,336: INFO: common: created directory at: artifacts]
[2024-07-28 11:53:07,336: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-28 11:53:07,336: INFO: 2991198467: Downloading data from https://drive.google.com/file/d/1WfP-M58gBkzCps5k5PE9g82u2mhiO8ur/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1WfP-M58gBkzCps5k5PE9g82u2mhiO8ur
To: c:\Personal Projects\NephroMassSorter\artifacts\data_ingestion\data.zip
100%|██████████| 19.7M/19.7M [00:00<00:00, 20.0MB/s]

[2024-07-28 11:53:12,755: INFO: 2991198467: Downloaded data from https://drive.google.com/file/d/1WfP-M58gBkzCps5k5PE9g82u2mhiO8ur/view?usp=sharing into file artifacts/data_ingestion/data.zip]
